In [1]:
import numpy as np
import math
from tabulate import tabulate
from chipsplitting import PSI_HAT_1_SUPP, PSI_HAT_2_SUPP, PSI_HAT_3_SUPP, PSI_HAT_4_SUPP, PSI_HAT_5_SUPP, PSI_HAT_6_SUPP, PHI_HAT_1_SUPP, PHI_HAT_2_SUPP, PHI_HAT_3_SUPP, PHI_HAT_4_SUPP, PHI_HAT_5_SUPP, PHI_HAT_6_SUPP, XI_HAT_1_EVEN_SUPP, XI_HAT_2_EVEN_SUPP, XI_HAT_3_EVEN_SUPP, XI_HAT_1_ODD_SUPP, XI_HAT_2_ODD_SUPP, XI_HAT_3_ODD_SUPP, XI_HAT_4_EVEN_SUPP, XI_HAT_4_ODD_SUPP, XI_HAT_5_EVEN_SUPP, XI_HAT_5_ODD_SUPP, XI_HAT_6_EVEN_SUPP, XI_HAT_6_ODD_SUPP

PSI_HAT_1_SUPP = np.array(PSI_HAT_1_SUPP[0]), np.array(PSI_HAT_1_SUPP[1])
PSI_HAT_2_SUPP = np.array(PSI_HAT_2_SUPP[0]), np.array(PSI_HAT_2_SUPP[1])
PSI_HAT_3_SUPP = np.array(PSI_HAT_3_SUPP[0]), np.array(PSI_HAT_3_SUPP[1])
PSI_HAT_4_SUPP = np.array(PSI_HAT_4_SUPP[0]), np.array(PSI_HAT_4_SUPP[1])
PSI_HAT_5_SUPP = np.array(PSI_HAT_5_SUPP[0]), np.array(PSI_HAT_5_SUPP[1])
PSI_HAT_6_SUPP = np.array(PSI_HAT_6_SUPP[0]), np.array(PSI_HAT_6_SUPP[1])
PHI_HAT_1_SUPP = np.array(PHI_HAT_1_SUPP[0]), np.array(PHI_HAT_1_SUPP[1])
PHI_HAT_2_SUPP = np.array(PHI_HAT_2_SUPP[0]), np.array(PHI_HAT_2_SUPP[1])
PHI_HAT_3_SUPP = np.array(PHI_HAT_3_SUPP[0]), np.array(PHI_HAT_3_SUPP[1])
PHI_HAT_4_SUPP = np.array(PHI_HAT_4_SUPP[0]), np.array(PHI_HAT_4_SUPP[1])
PHI_HAT_5_SUPP = np.array(PHI_HAT_5_SUPP[0]), np.array(PHI_HAT_5_SUPP[1])
PHI_HAT_6_SUPP = np.array(PHI_HAT_6_SUPP[0]), np.array(PHI_HAT_6_SUPP[1])

def ncr(n, r):
    f = math.factorial
    return f(n) // f(r) // f(n-r)

$n^2 +3n + 2 - 2m = 0$

$n = - \frac{3}{2} \pm \sqrt{\frac{1}{4} + 2m}$

In [2]:
def gauss(n):
    return int(n * (n+1) * 0.5)

def getIndex(col, row):
    return gauss(col + row) + col

def getMaxDegree(config):
    return int(-1.5 + np.sqrt(0.25 + 2*config.size))

def printConfig(config):
    max_deg = getMaxDegree(config)
    for len_row, row_index in enumerate(range(max_deg, -1, -1)):
        for col_index in range(len_row + 1):
            val = str(config[getIndex(col_index, row_index)])
            print(f"{''.join([' '] * (4 - len(val))) + val}", end=" ")
        print("\n", end="")
    print("")


def create_exhaust_generator(degree, support_size = 4):
    N = int((degree+1)*(degree+2)/2)
    for d in range(degree + 1):
        diag = getIndex(d, degree - d)
        for i in range(1, diag):
            for j in range(1, i):
                for k in range(1, j):
                    if diag != i and diag != j and diag != k:
                        yield (diag, i, j, k)

In [3]:
def hyperfield_add(A, B):
    res = np.sign(A + B)
    res[(A == -B) & (A != 0)] = np.nan
    res[np.isnan(A) | np.isnan(B)] = np.nan
    return res

def eval_f(support,x):
    support_pos, support_neg = support[0], support[1]
    if any(np.isnan(x[support_pos | support_neg])):
        return np.nan
    has_neg = any(x[support_pos] < 0) | any(x[support_neg] > 0)
    has_pos = any(x[support_pos] > 0) | any(x[support_neg] < 0)
    if has_neg and has_pos:
        return np.nan
    if has_neg:
        return -1
    elif has_pos:
        return 1
    return 0

### Lemma 6.9
$\mathrm{sign}(\varphi_{a,b})(x) = \sum^{a}_{i=0} \sum^{b}_{j=0} x_{i,j}$

In [4]:
def varphi(degree, a):
    support_pos = np.full(gauss(degree + 1), False)
    support_neg = np.full(gauss(degree + 1), False)
    for x in range(a+1):
        for y in range(degree - a, -1, -1):
            support_pos[getIndex(x,y)] = True
    return support_pos, support_neg

def psi(degree, k):
    support_pos = np.full(gauss(degree + 1), False)
    support_neg = np.full(gauss(degree + 1), False)
    for j in range(k+1):
        for i in range(k-j, degree-j + 1):
            if (k - j) % 2 == 0:
                support_pos[getIndex(i,j)] = True
            else:
                support_neg[getIndex(i,j)] = True
    return support_pos, support_neg

def psi_bar(degree, k):
    support_pos = np.full(gauss(degree + 1), False)
    support_neg = np.full(gauss(degree + 1), False)
    for j in range(k+1):
        for i in range(k-j, degree-j + 1):
            if (k - j) % 2 == 0:
                support_pos[getIndex(j,i)] = True
            else:
                support_neg[getIndex(j,i)] = True
    return support_pos, support_neg

Test the print function.

In [5]:
config = np.array([0,1,2,3,4,5,6,7,8,9], dtype="int")
printConfig(config)

   6 
   3    7 
   1    4    8 
   0    2    5    9 



## Explore pascal equations

The pascal equation $\psi_k$ is uniquely defined by the first column $e_k$.

In [6]:
degree = 13
# first column, k-th row is one
k = 10
config = np.full(gauss(degree + 1), 0)
supp_pos, supp_neg = psi_bar(degree, k)
config[supp_pos] = 1
config[supp_neg] = -1
printConfig(config)

   1 
   1   -1 
   1   -1    1 
   1   -1    1   -1 
   0   -1    1   -1    1 
   0    0    1   -1    1   -1 
   0    0    0   -1    1   -1    1 
   0    0    0    0    1   -1    1   -1 
   0    0    0    0    0   -1    1   -1    1 
   0    0    0    0    0    0    1   -1    1   -1 
   0    0    0    0    0    0    0   -1    1   -1    1 
   0    0    0    0    0    0    0    0    1   -1    1    0 
   0    0    0    0    0    0    0    0    0   -1    1    0    0 
   0    0    0    0    0    0    0    0    0    0    1    0    0    0 



## Count valid configurations

Goal: count the number of configurations $s \in H^{V_d}$ that are valid and have degree $deg \in \{2,3,..., 7 \}$.

In [49]:
table = [['support size', 'degree', 'total']]

support_size = 4
for deg in range(2, 14):
    count_diag = deg + 1
    count_lower_triangle = gauss(deg) - 1

    sum = 0
    # we start with 1 since every configuration should have degree deg
    for k in range(1, support_size +1):
        if count_diag >= k and count_lower_triangle >= support_size - k:
            sum += ncr(count_diag, k) * ncr(count_lower_triangle, support_size - k)
    table.append([support_size, deg, sum])

print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

╒════════════════╤══════════╤═════════╕
│   support size │   degree │   total │
╞════════════════╪══════════╪═════════╡
│              4 │        2 │       5 │
├────────────────┼──────────┼─────────┤
│              4 │        3 │     121 │
├────────────────┼──────────┼─────────┤
│              4 │        4 │     875 │
├────────────────┼──────────┼─────────┤
│              4 │        5 │    3844 │
├────────────────┼──────────┼─────────┤
│              4 │        6 │   12705 │
├────────────────┼──────────┼─────────┤
│              4 │        7 │   34810 │
├────────────────┼──────────┼─────────┤
│              4 │        8 │   83391 │
├────────────────┼──────────┼─────────┤
│              4 │        9 │  180500 │
├────────────────┼──────────┼─────────┤
│              4 │       10 │  360789 │
├────────────────┼──────────┼─────────┤
│              4 │       11 │  676235 │
├────────────────┼──────────┼─────────┤
│              4 │       12 │ 1201915 │
├────────────────┼──────────┼─────────┤


### Example deg = 2, support size = 4
Here we enumerate all valid configurations of degree $2$.

In [109]:
degree = 2
for supp in create_exhaust_generator(degree):
    w = np.array([1 if idx in supp else 0 for idx in range(gauss(degree+1))])
    printConfig(w)

| \
|1 \
|1 1 \
|0 1 0 \
- - - - 
| \
|1 \
|1 0 \
|0 1 1 \
- - - - 
| \
|0 \
|1 1 \
|0 1 1 \
- - - - 
| \
|1 \
|1 1 \
|0 0 1 \
- - - - 
| \
|1 \
|0 1 \
|0 1 1 \
- - - - 


### Example deg = 3, support size = 4
Here we enumerate all valid configurations of degree $3$.

In [103]:
degree = 3
for supp in create_exhaust_generator(degree):
    w = np.array([1 if idx in supp else 0 for idx in range(gauss(degree+1))])
    printConfig(w)

| \
|1 \
|1 0 \
|1 0 0 \
|0 1 0 0 \
- - - - - - 
| \
|1 \
|0 0 \
|1 1 0 \
|0 1 0 0 \
- - - - - - 
| \
|1 \
|1 0 \
|1 1 0 \
|0 0 0 0 \
- - - - - - 
| \
|1 \
|1 0 \
|0 1 0 \
|0 1 0 0 \
- - - - - - 
| \
|1 \
|0 0 \
|1 0 0 \
|0 1 1 0 \
- - - - - - 
| \
|1 \
|1 0 \
|1 0 0 \
|0 0 1 0 \
- - - - - - 
| \
|1 \
|1 0 \
|0 0 0 \
|0 1 1 0 \
- - - - - - 
| \
|1 \
|0 0 \
|1 1 0 \
|0 0 1 0 \
- - - - - - 
| \
|1 \
|0 0 \
|0 1 0 \
|0 1 1 0 \
- - - - - - 
| \
|1 \
|1 0 \
|0 1 0 \
|0 0 1 0 \
- - - - - - 
| \
|0 \
|1 1 \
|1 0 0 \
|0 1 0 0 \
- - - - - - 
| \
|0 \
|0 1 \
|1 1 0 \
|0 1 0 0 \
- - - - - - 
| \
|0 \
|1 1 \
|1 1 0 \
|0 0 0 0 \
- - - - - - 
| \
|0 \
|1 1 \
|0 1 0 \
|0 1 0 0 \
- - - - - - 
| \
|0 \
|0 1 \
|1 0 0 \
|0 1 1 0 \
- - - - - - 
| \
|0 \
|1 1 \
|1 0 0 \
|0 0 1 0 \
- - - - - - 
| \
|0 \
|1 1 \
|0 0 0 \
|0 1 1 0 \
- - - - - - 
| \
|0 \
|0 1 \
|1 1 0 \
|0 0 1 0 \
- - - - - - 
| \
|0 \
|0 1 \
|0 1 0 \
|0 1 1 0 \
- - - - - - 
| \
|0 \
|1 1 \
|0 1 0 \
|0 0 1 0 \
- - - - - - 
| \
|1 \
|0 1 \
|1 0

### Proof of Lemma 6.21 a,b,c

Goal: Find the number of valid configurations $s \in H^{V_d}$ with $|\mathrm{supp}^+(s)| = 4$ such that all basis systems $\varphi, \psi, \bar \psi$ vanish at $s$.

In [15]:
%%time
degree = 6
res = []
basis = [varphi(degree, a) for a in range(degree + 1)]
basis2 = [psi(degree, k) for k in range(degree + 1)]
basis3 = [psi_bar(degree, k) for k in range(degree + 1)]
for supp in create_exhaust_generator(degree):
    w = np.array([1 if idx in supp else 0 for idx in range(gauss(degree+1))])
    w[0] = -1
    if np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis))))) and np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis2))))) and np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis3))))):
        res.append(w)
print(f"Number of configurations: {len(res)}")
res

Number of configurations: 5
CPU times: user 702 ms, sys: 2.13 ms, total: 704 ms
Wall time: 706 ms


[array([-1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  1,  1,  0,  0,  1,  0,  0,  0]),
 array([-1,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  1,
         0,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0]),
 array([-1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  1,  0,  0,  1,  0,  0,  0,  0,  1]),
 array([-1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,
         0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  1]),
 array([-1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  0,  0,  1,  0,  0,  1])]

In [16]:
%%time
degree = 7
res = []
basis = [varphi(degree, a) for a in range(degree + 1)]
basis2 = [psi(degree, k) for k in range(degree + 1)]
basis3 = [psi_bar(degree, k) for k in range(degree + 1)]
for supp in create_exhaust_generator(degree):
    w = np.array([1 if idx in supp else 0 for idx in range(gauss(degree+1))])
    w[0] = -1
    if np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis))))) and np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis2))))) and np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis3))))):
        res.append(w)
print(f"Number of configurations: {len(res)}")
res

Number of configurations: 3
CPU times: user 2.01 s, sys: 2.54 ms, total: 2.01 s
Wall time: 2.02 s


[array([-1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,
         0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,
         0,  1]),
 array([-1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  1,  0,  0,  0,  0,  0,
         0,  1]),
 array([-1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0,
         0,  1])]

In [256]:
%%time
degree = 8
res = []
basis = [varphi(degree, a) for a in range(degree + 1)]
basis2 = [psi(degree, k) for k in range(degree + 1)]
basis3 = [psi_bar(degree, k) for k in range(degree + 1)]
for supp in create_exhaust_generator(degree):
    w = np.array([1 if idx in supp else 0 for idx in range(gauss(degree+1))])
    w[0] = -1
    if np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis))))) and np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis2))))) and np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis3))))):
        res.append(w)
print(f"Number of configurations: {len(res)}")

Number of configurations: 0
CPU times: user 5.2 s, sys: 3.33 ms, total: 5.2 s
Wall time: 5.23 s


In [259]:
%%time
degree = 9
res = []
basis = [varphi(degree, a) for a in range(degree + 1)]
basis2 = [psi(degree, k) for k in range(degree + 1)]
basis3 = [psi_bar(degree, k) for k in range(degree + 1)]
for supp in create_exhaust_generator(degree):
    w = np.array([1 if idx in supp else 0 for idx in range(gauss(degree+1))])
    w[0] = -1
    if np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis))))) and np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis2))))) and np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis3))))):
        res.append(w)
print(f"Number of configurations: {len(res)}")

Number of configurations: 0
CPU times: user 12.2 s, sys: 4.89 ms, total: 12.2 s
Wall time: 12.3 s


In [260]:
%%time
degree = 10
res = []
basis = [varphi(degree, a) for a in range(degree + 1)]
basis2 = [psi(degree, k) for k in range(degree + 1)]
basis3 = [psi_bar(degree, k) for k in range(degree + 1)]
for supp in create_exhaust_generator(degree):
    w = np.array([1 if idx in supp else 0 for idx in range(gauss(degree+1))])
    w[0] = -1
    if np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis))))) and np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis2))))) and np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis3))))):
        res.append(w)
print(f"Number of configurations: {len(res)}")

Number of configurations: 0
CPU times: user 26.9 s, sys: 10 ms, total: 26.9 s
Wall time: 27.1 s


In [261]:
%%time
degree = 11
res = []
basis = [varphi(degree, a) for a in range(degree + 1)]
basis2 = [psi(degree, k) for k in range(degree + 1)]
basis3 = [psi_bar(degree, k) for k in range(degree + 1)]
for supp in create_exhaust_generator(degree):
    w = np.array([1 if idx in supp else 0 for idx in range(gauss(degree+1))])
    w[0] = -1
    if np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis))))) and np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis2))))) and np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis3))))):
        res.append(w)
print(f"Number of configurations: {len(res)}")

Number of configurations: 0
CPU times: user 55.4 s, sys: 9.02 ms, total: 55.4 s
Wall time: 55.6 s



__Conclusion__: We conclude that only configurations $s$ of degree $d = 6$ or $d = 7$ belong to $\Omega_d$.

### Lemma 6.21 d

Fix some degree $d$.
We define $\hat \varphi_i, i=1,2,3$ to be the linear form of $\varphi_{i, d - i}$ in $H[x,y,z,b,c,d,e]$. A configuration $s$ in $H^{\Xi}$ uses the following coordinates $s = [ s_{i} ]_{i=0,...,59} = [x_0,...,x_{15}, y_{16},...,y_{31},z_{32},...,z_{47}, b_{48}, ..., b_{51}, c_{52},..., c_{55}, d_{56}, ..., d_{59}, e_{60},...,e_{63}] \in H^{64}$.

#### Find valid configurations for $d \geq 12$

Fix some degree $d$. We define the following sets of Pascal equations 
$$
    A = \{ \psi_1,..., \psi_6, \varphi_1, ...,  \varphi_6 \}
$$
where $\psi_1, \psi_2, \psi_3$ are the Pascal equations respectively defined by 
```
0                0                0
* .              * .              * .
0 . .            0 . .            1 . .
0 . . .          1 . . .          0 . . .
1 . . . .        0 . . . .        0 . . . .
0 . . . . .      0 . . . . .      0 . . . . .       * stands for arbitrarily many 0
```
$\psi_4, \psi_5, \psi_6$ are the Pascal equations respectively defined by 
```
.                .                .
. .              . .              . .
. . .            . . .            . . .
. . . .          . . . .          . . . .
. . . . .        . . . . .        . . . . . 
0 1 0 0 * 0      0 0 1 0 * 0      0 0 0 1 * 0       * stands for arbitrarily many 0
```
$\varphi_1,...,\varphi_6$ are the Pascal equations respectively defined by
```
0                0                0
. 1              . 0              . 0
. . 0            . . 1            . . 0
. . . 0          . . . 0          . . . 1
. . . . *        . . . . *        . . . . *
. . . . . 0      . . . . . 0      . . . . . 0      

0                0                0
. *              . *              . *
. . 1            . . 0            . . 0
. . . 0          . . . 1          . . . 0
. . . . 0        . . . . 0        . . . . 1
. . . . . 0      . . . . . 0      . . . . . 0       * stands for arbitrarily many 0
```
and $\xi_1,...,\xi_6$ are the Pascal equations respectively defined by
```
0                0                0
1 .              0 .              0 .
0 . .            1 . .            0 . .
0 . . .          0 . . .          1 . . .
* . . . .        * . . . .        * . . . . 
0 . . . . .      0 . . . . .      0 . . . . .      

.                .                .
. .              . .              . .
. . .            . . .            . . .
. . . .          . . . .          . . . .
. . . . .        . . . . .        . . . . . 
0 * 0 0 1 0      0 * 0 1 0 0      0 * 1 0 0 0       * stands for arbitrarily many 0
```

In [13]:
%%time
def countValidConfigsForContractions(degree="even"):
    assert degree == "even" or degree == "odd", "degree must be 'even' or 'odd'"
    
    def create_hat_exhaust_generator():
        # for d in range(63, 15, -1):
        for d in [19,23,27,31,56,60,47,46,45,44]:
            for i in range(1, 64):
                if i == d:
                    continue
                for j in range(1, i):
                    if j == d:
                        continue
                    for k in range(1, j):
                        if k == d:
                            continue
                        yield (d, i, j, k)
                    
    res = []
    A = np.array([
        PSI_HAT_1_SUPP, PSI_HAT_2_SUPP, PSI_HAT_3_SUPP, 
        PSI_HAT_4_SUPP, PSI_HAT_5_SUPP, PSI_HAT_6_SUPP, 
        PHI_HAT_1_SUPP, PHI_HAT_2_SUPP, PHI_HAT_3_SUPP, 
        PHI_HAT_4_SUPP, PHI_HAT_5_SUPP, PHI_HAT_6_SUPP
    ])
    EVEN = np.array([
        XI_HAT_1_EVEN_SUPP, XI_HAT_2_EVEN_SUPP, XI_HAT_3_EVEN_SUPP, 
        XI_HAT_4_EVEN_SUPP, XI_HAT_5_EVEN_SUPP, XI_HAT_6_EVEN_SUPP
    ])
    ODD = np.array([
        XI_HAT_1_ODD_SUPP, XI_HAT_2_ODD_SUPP, XI_HAT_3_ODD_SUPP, 
        XI_HAT_4_ODD_SUPP, XI_HAT_5_ODD_SUPP, XI_HAT_6_ODD_SUPP
    ])

    for supp in create_hat_exhaust_generator():
        w = np.array([1 if idx in supp else 0 for idx in range(64)])
        w[0] = -1
        
        is_valid = True
        for f in np.concatenate([A, EVEN if degree == "even" else ODD]):
            val = eval_f(f, w)
            if not np.isnan(val):
                is_valid = False
                break
    
        if is_valid:
            res.append(w)

    return res

res1 = countValidConfigsForContractions("even")
print(f"Number of configurations for even d: {len(res1)}")

res2 = countValidConfigsForContractions("odd")
print(f"Number of configurations for odd d: {len(res2)}")

Number of configurations for even d: 0
Number of configurations for odd d: 0
CPU times: user 11.4 s, sys: 43.3 ms, total: 11.5 s
Wall time: 11.5 s


**Conclusion:** For $d \geq 12$ there are no valid configurations in $H^{64}$ that satisfy all Pascal equations in $A \cup \mathrm{EVEN}$ or $A \cup \mathrm{ODD}$.

**Why is this good?** It is a necessary condition for valid configurations $w \in \mathbb Z^{V_d}$ that all Pascal equations vanish at $\mathrm{sign}(w)$. Since $A \cup \mathrm{EVEN}$ is a subset of all Pascal equations and $A \cup \mathrm{EVEN}$ have no zeros for configurations $s$ with support size $\leq 4$, no valid $w \in \mathbb Z^{V_{d}}$ exist for $d \geq 12$.

### Conclusion

With Lemma 6.21 a,b,c,d we see that valid configurations $s \in H^{V_d}$ with support size $4$ do not exist for degree $d \geq 8$. The next step is to show that for $d = 6$ and $d = 7$, the valid configurations $s \in H^{V_d}$ do not correspond to valid configurations $w \in \mathbb Z^{V_d}$. Thus, after showing this we can conclude that $|\mathrm{supp}^+(w)| = 4 \implies \mathrm{deg}(w) \leq 5$.